In [ ]:
import requests
import json
import pandas as pd
import time

API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = ""
MODEL_NAME = "mistral-small-latest"

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

INSTRUCTION = """
Ты — помощник по разметке вакансий. Если в тексте есть описание вакансии, извлеки следующие поля:
vacancy, company, city, country, experience, employment, schedule, salary_from, salary_to, currency, skills, responsibilities, requirements, experience level.

Верни ответ строго в формате JSON. Не добавляй пояснений, комментариев или текста вокруг.

Если зарплата в тексте не указана — верни salary_from и salary_to как null.
Если в тексте нет описания вакансии, верни null.
"""

def prepare_prompt(message: str) -> list:
    return [
        {"role": "system", "content": INSTRUCTION},
        {"role": "user", "content": f"Текст вакансии:\n{message}"}
    ]

def query_mistral(prompt: list, max_retries=3):
    for attempt in range(max_retries):
        try:
            response = requests.post(
                API_URL,
                headers=HEADERS,
                json={
                    "model": MODEL_NAME,
                    "messages": prompt,
                    "temperature": 0.2,
                    "max_tokens": 1224
                },
                timeout=120
            )
            response.raise_for_status()
            content = response.json()["choices"][0]["message"]["content"]
            content = content.replace("```json", "").replace("```", "").strip()

            if content.lower() == "null":
                return None

            return json.loads(content)

        except requests.exceptions.RequestException as e:
            print(f"Ошибка запроса (попытка {attempt+1}):", e)
            time.sleep(2)
        except json.JSONDecodeError as e:
            print("Ошибка парсинга JSON:", e)
            print("Сырой ответ:", content)
            return None

    return None

def process_vacancies(input_file, output_file):
    df = pd.read_csv(input_file)
    
    results = []

    for i, row in df.iterrows():
        print(f"Обработка вакансии {i+1}/{len(df)}...")
        prompt = prepare_prompt(row["text"])
        result = query_mistral(prompt)

        if result is None:
            results.append({})
        else:
            results.append({
                "vacancy": result.get("vacancy", ""),
                "company": result.get("company", ""),
                "city": result.get("city", ""),
                "country": result.get("country", ""),
                "experience": result.get("experience", ""),
                "employment": result.get("employment", ""),
                "schedule": result.get("schedule", ""),
                "salary_from": result.get("salary_from"),
                "salary_to": result.get("salary_to"),
                "currency": result.get("currency", ""),
                "skills": result.get("skills", []),
                "responsibilities": result.get("responsibilities", []),
                "requirements": result.get("requirements", []),
                "experience level": result.get("experience level", [])
            })

        time.sleep(2.5)  # 

    df_result = pd.concat([df, pd.DataFrame(results)], axis=1)
    df_result.to_csv(output_file, index=False)
    print(f"Готово! Результат сохранён в {output_file}")

In [ ]:
INPUT_CSV = "hh_text.csv"
OUTPUT_CSV = "hh_text_skills_mistral.csv"

process_vacancies(INPUT_CSV, OUTPUT_CSV)

Обработка вакансии 1/8187...
Обработка вакансии 2/8187...
Обработка вакансии 3/8187...
Обработка вакансии 4/8187...
Обработка вакансии 5/8187...
Обработка вакансии 6/8187...
Обработка вакансии 7/8187...
Обработка вакансии 8/8187...
Ошибка парсинга JSON: Unterminated string starting at: line 51 column 5 (char 3325)
Сырой ответ: {
  "vacancy": "Разработчик ИАС",
  "company": null,
  "city": null,
  "country": "Казахстан",
  "experience": null,
  "employment": "полная занятость",
  "schedule": "полный день",
  "salary_from": null,
  "salary_to": null,
  "currency": null,
  "skills": [
    "Big Data",
    "BI-инструменты (Power BI, Tableau, Grafana, Superset, QlikView)",
    "СУБД (SQL Server, Oracle, MySQL, MS SQL, PostgreSQL, NoSQL MongoDB, Cassandra)",
    "ETL-процессы (Talend, SSIS, Apache NiFi)",
    "языки программирования (Python, R)",
    "архитектура ИАС",
    "Agile, Scrum, Waterfall",
    "Camunda Modeler",
    "Postman, Swagger",
    "RESTful API, SOAP, JSON, XML",
    "BPMN 2

In [4]:
df_out = pd.read_csv('hh_text_skills_mistral.csv')


In [ ]:
#  все поля из text с промежуточным сохранением 

import requests
import json
import pandas as pd
import time
import os

API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = ""
MODEL_NAME = "mistral-small-latest"

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

INSTRUCTION = """
Ты — помощник по разметке вакансий. Если в тексте есть описание вакансии, извлеки следующие поля:
vacancy, company, city, country, experience, employment, schedule, salary_from, salary_to, currency, skills, responsibilities, requirements, experience level.

Верни ответ строго в формате JSON. Не добавляй пояснений, комментариев или текста вокруг.

Если зарплата в тексте не указана — верни salary_from и salary_to как null.
Если в тексте нет описания вакансии, верни null.
"""

def prepare_prompt(message: str) -> list:
    return [
        {"role": "system", "content": INSTRUCTION},
        {"role": "user", "content": f"Текст вакансии:\n{message}"}
    ]

def query_mistral(prompt: list, max_retries=3):
    for attempt in range(max_retries):
        try:
            response = requests.post(
                API_URL,
                headers=HEADERS,
                json={
                    "model": MODEL_NAME,
                    "messages": prompt,
                    "temperature": 0.2,
                    "max_tokens": 1224
                },
                timeout=120
            )
            response.raise_for_status()
            content = response.json()["choices"][0]["message"]["content"]
            content = content.replace("```json", "").replace("```", "").strip()

            if content.lower() == "null":
                return None

            return json.loads(content)

        except requests.exceptions.RequestException as e:
            print(f"Ошибка запроса (попытка {attempt+1}):", e)
            time.sleep(2)
        except json.JSONDecodeError as e:
            print("Ошибка парсинга JSON:", e)
            print("Сырой ответ:", content)
            return None

    return None

def process_vacancies(input_file, output_file, partial_file='output_partial_2.csv', id_column='ID'):
    df = pd.read_csv(input_file)

    # Восстановление прогресса
    if os.path.exists(partial_file):
        df_partial = pd.read_csv(partial_file)
        processed_ids = set(df_partial[id_column])
        print(f"Найден частичный файл. Уже обработано: {len(processed_ids)} вакансий.")
    else:
        df_partial = pd.DataFrame()
        processed_ids = set()

    for i, row in df.iterrows():
        vacancy_id = row[id_column]
        if vacancy_id in processed_ids:
            continue

        print(f"Обработка вакансии {i+1}/{len(df)} (ID: {vacancy_id})...")
        prompt = prepare_prompt(row["text"])
        result = query_mistral(prompt)

        if result is None:
            row_result = {
                id_column: vacancy_id,
                "vacancy": "", "company": "", "city": "", "country": "",
                "experience": "", "employment": "", "schedule": "",
                "salary_from": None, "salary_to": None, "currency": "",
                "skills": [], "responsibilities": [], "requirements": [], "experience level": []
            }
        else:
            row_result = {
                id_column: vacancy_id,
                "vacancy": result.get("vacancy", ""),
                "company": result.get("company", ""),
                "city": result.get("city", ""),
                "country": result.get("country", ""),
                "experience": result.get("experience", ""),
                "employment": result.get("employment", ""),
                "schedule": result.get("schedule", ""),
                "salary_from": result.get("salary_from"),
                "salary_to": result.get("salary_to"),
                "currency": result.get("currency", ""),
                "skills": result.get("skills", []),
                "responsibilities": result.get("responsibilities", []),
                "requirements": result.get("requirements", []),
                "experience level": result.get("experience level", [])
            }

        # Сохраняем результат построчно
        df_partial = pd.concat([df_partial, pd.DataFrame([row_result])], ignore_index=True)
        df_partial.to_csv(partial_file, index=False)

        time.sleep(2.5)  # таймаут между запросами

    # Финальное сохранение
    df_merged = pd.merge(df, df_partial, on=id_column, how="left")
    df_merged.to_csv(output_file, index=False)
    print(f"Готово! Финальный результат сохранён в {output_file}")


In [10]:
INPUT_CSV = "hh_skills_to_check_2.csv"
OUTPUT_CSV = "hh_skills_checked_2.csv"

process_vacancies(INPUT_CSV, OUTPUT_CSV)

Обработка вакансии 1/14203 (ID: nan)...
Обработка вакансии 2/14203 (ID: 120633960.0)...
Обработка вакансии 3/14203 (ID: 121320799.0)...
Обработка вакансии 4/14203 (ID: 121234224.0)...
Обработка вакансии 5/14203 (ID: 120878881.0)...
Обработка вакансии 6/14203 (ID: 121362399.0)...
Обработка вакансии 7/14203 (ID: 121086553.0)...
Обработка вакансии 8/14203 (ID: 120845093.0)...
Обработка вакансии 9/14203 (ID: 120430920.0)...
Обработка вакансии 10/14203 (ID: 120207253.0)...
Обработка вакансии 11/14203 (ID: 121353288.0)...
Обработка вакансии 12/14203 (ID: 121438321.0)...
Обработка вакансии 13/14203 (ID: 118827760.0)...
Обработка вакансии 14/14203 (ID: 121098312.0)...
Обработка вакансии 15/14203 (ID: 120640605.0)...
Обработка вакансии 16/14203 (ID: 120706524.0)...
Обработка вакансии 17/14203 (ID: 120703453.0)...
Обработка вакансии 18/14203 (ID: 121038064.0)...
Обработка вакансии 19/14203 (ID: 121364092.0)...
Обработка вакансии 20/14203 (ID: 120075925.0)...
Обработка вакансии 21/14203 (ID: 1213

In [ ]:
# только з/п

import requests
import json
import pandas as pd
import time

API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = ""
MODEL_NAME = "mistral-small-latest"

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

INSTRUCTION = """
Ты — помощник по разметке вакансий. Если в тексте есть описание вакансии, извлеки следующие поля:
salary from, salary to.

Верни ответ строго в формате JSON. Не добавляй пояснений, комментариев или текста вокруг.

Если зарплата в тексте не указана — верни salary from и salary to как null.
"""

def prepare_prompt(message: str) -> list:
    return [
        {"role": "system", "content": INSTRUCTION},
        {"role": "user", "content": f"Текст вакансии:\n{message}"}
    ]

def query_mistral(prompt: list, max_retries=3):
    for attempt in range(max_retries):
        try:
            response = requests.post(
                API_URL,
                headers=HEADERS,
                json={
                    "model": MODEL_NAME,
                    "messages": prompt,
                    "temperature": 0.2,
                    "max_tokens": 1024
                },
                timeout=120
            )
            response.raise_for_status()
            content = response.json()["choices"][0]["message"]["content"]
            content = content.replace("```json", "").replace("```", "").strip()

            if content.lower() == "null":
                return None

            return json.loads(content)

        except requests.exceptions.RequestException as e:
            print(f"Ошибка запроса (попытка {attempt+1}):", e)
            time.sleep(2)
        except json.JSONDecodeError as e:
            print("Ошибка парсинга JSON:", e)
            print("Сырой ответ:", content)
            return None

    return None

def process_vacancies(input_file, output_file):
    df = pd.read_csv(input_file)
    #df = df.iloc[50:]  # Для теста
    results = []

    for i, row in df.iterrows():
        print(f"Обработка вакансии {i+1}/{len(df)}...")
        prompt = prepare_prompt(row["text"])
        result = query_mistral(prompt)

        if result is None:
            results.append({})
        else:
            results.append({

                "salary_from": result.get("salary from"),
                "salary_to": result.get("salary to")

            })

        time.sleep(2)  # 

    df_result = pd.concat([df, pd.DataFrame(results)], axis=1)
    df_result.to_csv(output_file, index=False)
    print(f"Готово! Результат сохранён в {output_file}")

In [ ]:
INPUT_CSV = "df_no_salary.csv"
OUTPUT_CSV = "df_no_salary_mistral.csv"

process_vacancies(INPUT_CSV, OUTPUT_CSV)

In [4]:
# код с промежуточным сохранением 

def process_vacancies(input_file, output_file, partial_file='output_partial.csv'):
    df = pd.read_csv(input_file)

    # Если уже есть частичный файл, восстановим прогресс
    try:
        df_partial = pd.read_csv(partial_file)
        processed_ids = set(df_partial['ID'])
        print(f"Продолжаем с {len(processed_ids)} уже обработанных вакансий.")
    except FileNotFoundError:
        df_partial = pd.DataFrame()
        processed_ids = set()

    results = []

    for i, row in df.iterrows():
        if row['ID'] in processed_ids:
            continue

        print(f"Обработка вакансии {i+1}/{len(df)} ID={row['ID']}...")
        prompt = prepare_prompt(row["text"])
        result = query_mistral(prompt)

        row_result = row.to_dict()
        if result is None:
            row_result.update({
                "salary_from": None,
                "salary_to": None
            })
        else:
            row_result.update({
                "salary_from": result.get("salary from"),
                "salary_to": result.get("salary to")
            })

        results.append(row_result)

        # Промежуточное сохранение после каждой вакансии
        df_partial = pd.concat([df_partial, pd.DataFrame([row_result])], ignore_index=True)
        df_partial.to_csv(partial_file, index=False)

        time.sleep(1.5)

    # Финальное сохранение
    df_final = pd.read_csv(partial_file)
    df_final.to_csv(output_file, index=False)
    print(f"Готово! Полный результат сохранён в {output_file}")


In [5]:
INPUT_CSV = "df_no_salary.csv"
OUTPUT_CSV = "df_no_salary_mistral.csv"

process_vacancies(INPUT_CSV, OUTPUT_CSV)

Обработка вакансии 1/23162 ID=121356737.0...
Обработка вакансии 2/23162 ID=118037912.0...
Обработка вакансии 3/23162 ID=121304481.0...
Обработка вакансии 4/23162 ID=121670814.0...
Обработка вакансии 5/23162 ID=120258113.0...
Обработка вакансии 6/23162 ID=121382544.0...
Обработка вакансии 7/23162 ID=121229312.0...
Обработка вакансии 8/23162 ID=122027058.0...
Обработка вакансии 9/23162 ID=121917250.0...
Обработка вакансии 10/23162 ID=122069294.0...
Обработка вакансии 11/23162 ID=121399785.0...
Обработка вакансии 12/23162 ID=121488199.0...
Обработка вакансии 13/23162 ID=121510800.0...
Обработка вакансии 14/23162 ID=122010946.0...
Обработка вакансии 15/23162 ID=119900348.0...
Обработка вакансии 16/23162 ID=121885341.0...
Обработка вакансии 17/23162 ID=121837576.0...
Обработка вакансии 18/23162 ID=122206008.0...
Обработка вакансии 19/23162 ID=121239769.0...
Обработка вакансии 20/23162 ID=119954975.0...
Обработка вакансии 21/23162 ID=120835797.0...
Обработка вакансии 22/23162 ID=121804872.0.

In [ ]:
# exp

import pandas as pd
import requests
import json
import time
import random
from tqdm import tqdm

API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = ""
MODEL_NAME = "mistral-small-latest"
MAX_RETRIES = 3
BATCH_SIZE = 15
REQUEST_DELAY = 5

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

INSTRUCTION = """
Ты помощник по извлечению опыта работы из вакансий. Для каждого текста определи, сколько у кандидата должен быть опыт работы, и заполни поле `experience` одним из следующих значений:

- "Нет опыта"
- "1–3 года"
- "3–6 лет"
- "Более 6 лет"

Если информация об опыте не указана, поставь null.

Ответ строго в формате JSON:
{
  "results": [
    {"experience": "Нет опыта" / "1–3 года" / "3–6 лет" / "Более 6 лет" / null},
    ...
  ]
}
"""

def prepare_prompt(batch):
    entries = "\n".join([f"{i+1}. {text}" for i, text in enumerate(batch)])
    return [
        {"role": "system", "content": INSTRUCTION},
        {"role": "user", "content": f"Вот тексты вакансий:\n{entries}\n\nОтвет строго в JSON по формату выше."}
    ]

def query_mistral(prompt, max_tokens=1024):
    for attempt in range(MAX_RETRIES):
        try:
            response = requests.post(
                API_URL,
                headers=HEADERS,
                json={
                    "model": MODEL_NAME,
                    "messages": prompt,
                    "temperature": 0.1,
                    "max_tokens": max_tokens,
                    "response_format": {"type": "json_object"}
                },
                timeout=120
            )
            response.raise_for_status()
            content = response.json()["choices"][0]["message"]["content"].strip()

            if content.startswith("```json"):
                content = content[7:-3].strip()

            content = content.replace("\n", " ").replace("\r", "")
            content = content.replace(",}", "}").replace(",]", "]")

            return json.loads(content)

        except (requests.exceptions.RequestException, json.JSONDecodeError, KeyError) as e:
            print(f"\nОшибка (попытка {attempt+1}): {str(e)}")
            try:
                print("Сырой ответ:\n", response.text)
            except:
                pass
            time.sleep(2 + random.random() * 3)

    return None

def extract_experience(input_file):
    df = pd.read_csv(input_file)
    if 'experience' not in df.columns:
        df['experience'] = None

    texts = df['text'].fillna("").astype(str).tolist()

    for i in tqdm(range(0, len(df), BATCH_SIZE)):
        batch_texts = texts[i:i+BATCH_SIZE]
        prompt = prepare_prompt(batch_texts)
        response = query_mistral(prompt)

        if response and 'results' in response:
            for j, item in enumerate(response['results'][:len(batch_texts)]):
                idx = i + j
                df.at[idx, 'experience'] = item.get('experience', None)
        else:
            print(f"⚠️ Пропущен батч с {i} по {i + BATCH_SIZE}")

        df.to_csv(input_file, index=False)

        if i + BATCH_SIZE < len(df):
            time.sleep(REQUEST_DELAY + random.random())

    df.to_csv(input_file, index=False)
    print(f"\n✅ Извлечение завершено. Результаты сохранены в: {input_file}")
    return df

# Запуск
if __name__ == "__main__":
    extract_experience("df_to_ml_experience.csv")




In [ ]:
# country 

import pandas as pd
import requests
import json
import time
import random
from tqdm import tqdm

API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = "1FyhNdyIvR4hR29waKXtRf4NaGAwl5Ru"
MODEL_NAME = "mistral-small-latest"
MAX_RETRIES = 3
BATCH_SIZE = 30
REQUEST_DELAY = 4

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

INSTRUCTION = """
Ты географический помощник. По каждому названию города определи страну, в которой он находится.

Ответ строго в формате JSON:
{
  "results": [
    {"country": "Россия"},
    ...
  ]
}

Если город не удаётся распознать или страна неизвестна — верни null.
"""

def prepare_prompt(batch):
    cities = "\n".join([f"{i+1}. {city}" for i, city in enumerate(batch)])
    return [
        {"role": "system", "content": INSTRUCTION},
        {"role": "user", "content": f"Определи страну для следующих городов:\n{cities}\n\nОтвет строго в JSON по формату выше."}
    ]

def query_mistral(prompt, max_tokens=400):
    for attempt in range(MAX_RETRIES):
        try:
            response = requests.post(
                API_URL,
                headers=HEADERS,
                json={
                    "model": MODEL_NAME,
                    "messages": prompt,
                    "temperature": 0.1,
                    "max_tokens": max_tokens,
                    "response_format": {"type": "json_object"}
                },
                timeout=120
            )
            response.raise_for_status()
            content = response.json()["choices"][0]["message"]["content"].strip()

            if content.startswith("```json"):
                content = content[7:-3].strip()

            content = content.replace("\n", " ").replace("\r", "")
            content = content.replace(",}", "}").replace(",]", "]")

            return json.loads(content)

        except (requests.exceptions.RequestException, json.JSONDecodeError, KeyError) as e:
            print(f"\nОшибка (попытка {attempt+1}): {str(e)}")
            try:
                print("Сырой ответ:\n", response.text)
            except:
                pass
            time.sleep(2 + random.random() * 3)

    return None

def extract_country(input_file):
    df = pd.read_csv(input_file)
    if 'country' not in df.columns:
        df['country'] = None

    # Берём только строки с непустыми city и пустыми country
    mask = df['city'].notna() & (df['city'].astype(str).str.strip() != "") & df['country'].isna()
    target_df = df[mask].copy()

    cities = target_df['city'].astype(str).tolist()
    index_list = target_df.index.tolist()

    for i in tqdm(range(0, len(cities), BATCH_SIZE)):
        batch_cities = cities[i:i+BATCH_SIZE]
        batch_indices = index_list[i:i+BATCH_SIZE]
        prompt = prepare_prompt(batch_cities)
        response = query_mistral(prompt)

        if response and 'results' in response:
            for j, item in enumerate(response['results'][:len(batch_cities)]):
                idx = batch_indices[j]
                df.at[idx, 'country'] = item.get('country', None)
        else:
            print(f"⚠️ Пропущен батч с {i} по {i + BATCH_SIZE}")

        df.to_csv(input_file, index=False)

        if i + BATCH_SIZE < len(cities):
            time.sleep(REQUEST_DELAY + random.random())

    df.to_csv(input_file, index=False)
    print(f"\n✅ Определение стран завершено. Результаты сохранены в: {input_file}")
    return df

# Запуск
if __name__ == "__main__":
    extract_country("city_country_3.csv")



In [ ]:
# contry 2 с защитой 

import requests
import json
import pandas as pd
import time
import random
from tqdm import tqdm

API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = "1FyhNdyIvR4hR29waKXtRf4NaGAwl5Ru"
MODEL_NAME = "mistral-small-latest"
MAX_RETRIES = 3
BATCH_SIZE = 20
REQUEST_DELAY = 4
INPUT_FILE = "df_city_country.csv"

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

INSTRUCTION = """
Ты географический помощник. По каждому названию города определи страну, в которой он находится.

Ответ строго в формате JSON:
{
  "results": [
    {"country": "Россия"},
    ...
  ]
}

Если город не удаётся распознать или страна неизвестна — верни null.
"""

def prepare_prompt(batch):
    cities = "\n".join([f"{i+1}. {city}" for i, city in enumerate(batch)])
    return [
        {"role": "system", "content": INSTRUCTION},
        {"role": "user", "content": f"Определи страну для следующих городов:\n{cities}\n\nОтвет строго в JSON по формату выше."}
    ]

def query_mistral(prompt, max_tokens=500):
    for attempt in range(MAX_RETRIES):
        try:
            response = requests.post(
                API_URL,
                headers=HEADERS,
                json={
                    "model": MODEL_NAME,
                    "messages": prompt,
                    "temperature": 0.1,
                    "max_tokens": max_tokens,
                    "response_format": {"type": "json_object"}
                },
                timeout=120
            )
            response.raise_for_status()
            content = response.json()["choices"][0]["message"]["content"].strip()

            if content.startswith("```json"):
                content = content[7:-3].strip()

            content = content.replace("\n", " ").replace("\r", "")
            content = content.replace(",}", "}").replace(",]", "]")

            return json.loads(content)

        except (requests.exceptions.RequestException, json.JSONDecodeError, KeyError) as e:
            print(f"\nОшибка (попытка {attempt+1}): {str(e)}")
            try:
                print("Сырой ответ:\n", response.text)
            except:
                pass
            time.sleep(2 + random.random() * 3)

    return None

def extract_country(input_file):
    print("🔄 Загрузка текущего состояния файла...")
    df = pd.read_csv(input_file)
    if 'country' not in df.columns:
        df['country'] = None

    # Целевые строки: city непустой и country ещё не указан
    mask = df['city'].notna() & (df['city'].astype(str).str.strip() != "") & df['country'].isna()
    target_df = df[mask].copy()

    cities = target_df['city'].astype(str).tolist()
    index_list = target_df.index.tolist()

    print(f"🔍 Найдено {len(cities)} городов без страны. Начинаем обработку...")

    for i in tqdm(range(0, len(cities), BATCH_SIZE)):
        # Загрузка последней версии файла (на случай падения)
        df = pd.read_csv(input_file)

        batch_cities = cities[i:i + BATCH_SIZE]
        batch_indices = index_list[i:i + BATCH_SIZE]

        # Пропустить батч, если все страны уже определены
        already_done = all(pd.notna(df.loc[idx, 'country']) for idx in batch_indices)
        if already_done:
            continue

        prompt = prepare_prompt(batch_cities)
        response = query_mistral(prompt)

        if response and 'results' in response:
            for j, item in enumerate(response['results'][:len(batch_cities)]):
                idx = batch_indices[j]
                df.at[idx, 'country'] = item.get('country', None)
        else:
            print(f"⚠️ Пропущен батч с {i} по {i + BATCH_SIZE}")

        df.to_csv(input_file, index=False)

        if i + BATCH_SIZE < len(cities):
            time.sleep(REQUEST_DELAY + random.random())

    print(f"\n✅ Обработка завершена. Файл сохранён: {input_file}")
    return df

# Запуск
if __name__ == "__main__":
    extract_country(INPUT_FILE)